In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)
pd.set_option("display.max_colwidth", 100)

products_cl = pd.read_csv('products_cl.csv')
orders_qu = pd.read_csv('orders_qu.csv')
orderlines_qu = pd.read_csv('orderlines_qu.csv')
brands_cl = pd.read_csv('brands_cl.csv')

# url = "https://drive.google.com/file/d/15q4B4Mgl7zSRRNSHgt0fo7Hn2GDUNx_h/view?usp=sharing" # products.csv
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# products_cl = pd.read_csv(path)

# url = "https://drive.google.com/file/d/16MehTZmz2eI0tUYgtNKXg1aJzjU-RNvo/view?usp=sharing" # orders.csv
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# orders_qu = pd.read_csv(path)

# url = "https://drive.google.com/file/d/1JGS6vrvo6tLNj7oQMOikcrU_WlHFnlNj/view?usp=sharing" # orderlines.csv
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# orderlines_qu = pd.read_csv(path)

# url = "https://drive.google.com/file/d/1pAjJqtC_rrYTZ79bemuffh__KNOXhOyl/view?usp=sharing" # brands.csv
# path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
# brands_cl = pd.read_csv(path)


# backup
# products_df = products.copy()
# orders_df = orders.copy()
# orderlines_df = orderlines.copy()
# brands_df = brands.copy()

# products https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=drive_link
# orders https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=drive_link
# orderlines https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=drive_link
# brands https://drive.google.com/file/d/1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD/view?usp=drive_link

# orders_qu https://drive.google.com/file/d/16MehTZmz2eI0tUYgtNKXg1aJzjU-RNvo/view?usp=sharing
# orderlines_qu https://drive.google.com/file/d/1JGS6vrvo6tLNj7oQMOikcrU_WlHFnlNj/view?usp=sharing
# products_cl https://drive.google.com/file/d/15q4B4Mgl7zSRRNSHgt0fo7Hn2GDUNx_h/view?usp=sharing
# brands_cl https://drive.google.com/file/d/1pAjJqtC_rrYTZ79bemuffh__KNOXhOyl/view?usp=sharing

- What is the time period that the dataset covers?
- What is the overall revenue for that time?
- Are there seasonal patterns in the evolution of sales?
- What are the most sold products?
- What are the products that generate the most revenue?

In [20]:
# data preparation
orders_qu['created_date'] = pd.to_datetime(orders_qu['created_date'])
orderlines_qu['date'] = pd.to_datetime(orderlines_qu['date'])
orderlines_qu.loc[:,'unit_price_total'] = orderlines_qu['unit_price'] * orderlines_qu['product_quantity']

## Category

In [36]:
products_category_df = products_cl.copy()

New dataframe 'final_df'

key: type

In [37]:
# Convert columns to lowercase
products_category_df['name_lower'] = products_category_df['name'].str.lower()
products_category_df['desc_lower'] = products_category_df['desc'].str.lower()

# Remove whitespace before " and add a single space after "
products_category_df['name_lower'] = products_category_df['name_lower'].str.replace(r'\s+"', '" ', regex=True)
products_category_df['desc_lower'] = products_category_df['desc_lower'].str.replace(r'\s+"', '" ', regex=True)

# Combine words from 'name' and 'desc' columns into one column, split and explode
combined_words = products_category_df.assign(
    words=products_category_df['name_lower'].str.split() + products_category_df['desc_lower'].str.split()
).explode('words')

# Filter out unwanted standalone symbols (/, |, -, +) using regex
combined_words = combined_words[~combined_words['words'].str.match(r'^[\|\-/+]$')]

# Group by 'type' and 'words' to count word occurrences
word_counts = (
    combined_words
    .groupby(['type', 'words'])
    .size()
    .reset_index(name='count')
    .sort_values(['type', 'count'], ascending=[True, False])
)

# Get the top 8 words for each 'type'
top_words = (
    word_counts
    .groupby('type')
    .apply(lambda x: x.nlargest(8, 'count'))
    .reset_index(drop=True)
)

# Aggregate results to get the final DataFrame
final_df = (
    top_words
    .groupby('type')
    .agg(
        count=('count', 'sum'),  # Sum of counts of top 3 words
        top8_words=('words', lambda x: ', '.join(x))  # Comma-separated top 3 words
    )
    .reset_index()
)
final_df.head(3)


,type,count,top8_words
0,"1,02E+12",1768,"pro, macbook, ssd, core, bar, touch, 16gb, i7"
1,"1,44E+11",1152,"repair, iphone, and, parts, labor, for, service, including"
2,10142,178,"battery, macbook, pro, newertech, nupower, late, 13"", air"


In [38]:
# Grouping the products_category_df table by type
type_price_info = (
    products_category_df
    .groupby('type')
    .agg(
        sku_count=('sku', 'count'),
        price_avg=('price', 'mean'),  
        price_median=('price', 'median'), 
        price_min=('price', 'min'),
        price_max=('price', 'max')
    )
    .reset_index()
)

# Merging tables with prices and the top 8 words
type_price_top8 = type_price_info.merge(final_df, how='left', on='type')

# Joining products with order lines by SKU
product_orders_info = products_category_df.merge(orderlines_qu.loc[:,['sku','unit_price_total','product_quantity','id_order']], how='left', on='sku')

# Grouping by type to calculate the aggregated sum of unit prices
type_orders_info = product_orders_info.groupby('type').agg(unit_price_total=('unit_price_total', 'sum'), unique_orders=('id_order', 'nunique'), product_quantity=('product_quantity','sum')).reset_index()
type_orders_info = type_orders_info.sort_values('unit_price_total', ascending=False)

# Final table with information about types
type_info = type_price_top8.merge(type_orders_info, how='left', on='type')
type_info_df = type_info.copy()


Now we can manually assign a category for most types.

example:
```
{
'5,74E+15':'imac',
'1,02E+12':'macbook',
'1282':'imac',
'11935397':'hard drive',
...
}
```

In [39]:
type_info.sort_values('unit_price_total', ascending=False).head(5)

,type,sku_count,price_avg,price_median,price_min,price_max,count,top8_words,unit_price_total,unique_orders,product_quantity
17,11935397,562,751.52,211.49,8.99,10212.99,4316,"and, hard, drive, usb, mac, 3.0, for, external",738576.55,3669,4324.00
40,1296,187,512.68,399.00,104.99,2328.04,1188,"monitor, and, for, mac, hdmi, pc, with, ips",656947.12,1471,1722.00
23,12175397,939,1203.41,1006.95,91.99,5894.49,6944,"nas, and, server, ram, mac, synology, for, seagate",460434.85,1085,1114.00
24,12215397,170,404.09,279.00,49.90,2660.99,1144,"ssd, and, macbook, for, pro, mac, kit, owc",451277.53,1927,2322.00
49,1364,216,179.99,99.99,10.99,1383.96,1483,"mac, memory, ram, so-dimm, ddr3, pro, imac, owc",387748.96,3030,3822.00


In [40]:
type_dict = {
    '5,74E+15':'Apple iMac 27',
    '1,02E+12':'Apple Macbook Pro 13/15',
    '1282':'Apple iMac',
    '11935397':'Hard drive',
    '1296':'Monitor',
    '12175397':'NAS Server / Network',
    '2,17E+11':'(Open) Apple Macbook Pro 13/ 15',
    '12215397':'SSD',
    '1405':'graphic tablet',
    '2,16E+11':'Apple iMac 21.5',
    '1364':'mac RAM',
    '12655397':'Mac PC Hard Drive',
    '2158':'Apple MacBook Pro',
    '51601716':'(Open) Apple iPhone',
    '11905404':'bluetooth device',
    '113281716':'Apple iPhone 8 Plus',
    '5384':'Headphones',
    '113291716':'Apple iPhone 8',
    '85641716':'Apple iPhone 7',
    '85651716':'Apple iPhone 7 Plus',
    '106431714':'Apple iPad 64gb',
    '1433':'SSD',
    '24895185':'Apple Watch',
    '1298':'Open',
    '5398':'speaker',
    '1714':'iPad',
    '113271716':'Apple iPhone X',
    '21561716':'Apple iPhone 6',
    '12585395':'Adapter',
    '11865403':'iPhone case',
    '1231':'Service',
    '21632158':'mac',
    '5,39E+11':'macbook',
    '12995397':'thunderbolt',
    '10142':'battery',
    '13855401':'keyboard',
    '51861714':'iPad pro 256gb',
    '24821716':'iphone 6s',
    '118692158':'imac',
    '13005399':'macbook charger',
    '24885185':'apple watch',
    '1229':'pointer',
    '42945397':'lightning usb',
    '12635403':'iPad case',
    '51871714':'ipad',
    '1334':'router',
    '57445397':'memory card',
    '1325':'cable',
    '1387':'mouse',
    '13835403':'macbook case',
    '2449':'Apple Watch'
}
products_category_df['category'] = products_category_df['type'].map(type_dict)
products_category_df.loc[products_category_df['category'].isna(),'category'] = 'other'

In [41]:
products_category_df.groupby('category').size().sort_values(ascending=False)

category
other                              1711
iPhone case                        1057
NAS Server / Network                939
Open                                783
Hard drive                          562
bluetooth device                    454
Apple iMac                          373
iPad case                           362
SSD                                 341
macbook case                        269
Apple iMac 27                       247
mac RAM                             216
Adapter                             190
Monitor                             187
cable                               183
Headphones                          178
Apple Watch                         174
speaker                             159
Apple Macbook Pro 13/15             130
memory card                         129
router                              115
Apple MacBook Pro                   107
Mac PC Hard Drive                   105
pointer                             104
thunderbolt                    

In [29]:
products_category_df.loc[products_category_df['type'] == '12755395'].sort_values('price', ascending=True).sample(5)

,sku,name,desc,price,in_stock,type,name_lower,desc_lower,category
92,OWC0008,OWC Data Doubler Optical Bay Adapter Mac Mini (2009) iMac (2009-11),Tray substitution of superdrive by hard drive 25 inches.,72.99,1,12755395,owc data doubler optical bay adapter mac mini (2009) imac (2009-11),tray substitution of superdrive by hard drive 25 inches.,other
279,OWC0054,"OWC SSD installation Kit for iMac 27 ""(2010) without tools",SSD installation kit without tools iMac 27-inch (model 2010),35.99,0,12755395,"owc ssd installation kit for imac 27"" (2010) without tools",ssd installation kit without tools imac 27-inch (model 2010),other
2276,OWC0157,OWC Data Doubler Optical Bay Adapter Mac Mini (2010),Tray substitution of superdrive by hard drive 25 inches.,72.99,0,12755395,owc data doubler optical bay adapter mac mini (2010),tray substitution of superdrive by hard drive 25 inches.,other
39,OWC0007,OWC Data Doubler Optical Bay adapter Macbook / Macbook Pro,Tray SuperDrive replacement hard disk or SSD.,72.99,1,12755395,owc data doubler optical bay adapter macbook / macbook pro,tray superdrive replacement hard disk or ssd.,other
904,OWC0117,"OWC Multi-Mount Kit Mac Pro Adapter 2.5 ""/3.5"" /5.25 """,Adapter hard drive or SSD 2.5-inch to 3.5 inches and 3.5 inches to 5.25 inches for Mac Pro,54.99,0,12755395,"owc multi-mount kit mac pro adapter 2.5"" /3.5"" /5.25""",adapter hard drive or ssd 2.5-inch to 3.5 inches and 3.5 inches to 5.25 inches for mac pro,other
